In [1]:
import pandas as pd

In [2]:
df1=pd.read_csv('new_order.csv')

In [3]:
df1=df1[df1['sale_ord_dt']>'2019-12-31']
df1=df1[df1['sale_ord_dt']<'2024-01-01']

In [4]:
df1=df1[df1['rev_addr_province_id']!='台湾']
df1=df1[df1['rev_addr_province_id']!='港澳']

In [5]:
df1 = df1[df1['year'] != 2023]
df1

,sale_ord_dt,rev_addr_province_id,rev_addr_city_id,item_first_cate_name,year,month,season,nums
7668203,2020-01-01,上海,嘉定区,个人护理,2020,1,1,2918
7668204,2020-01-01,上海,嘉定区,个护美妆,2020,1,1,37
7668205,2020-01-01,上海,嘉定区,二手商品,2020,1,1,91
7668206,2020-01-01,上海,嘉定区,京东服务,2020,1,1,9
7668207,2020-01-01,上海,嘉定区,休闲零食,2020,1,1,48
...,...,...,...,...,...,...,...,...
30648961,2022-12-31,黑龙江,齐齐哈尔市,院端医疗器械,2022,12,4,3
30648962,2022-12-31,黑龙江,齐齐哈尔市,鞋包服饰/百货,2022,12,4,4
30648963,2022-12-31,黑龙江,齐齐哈尔市,鞋靴,2022,12,4,98
30648964,2022-12-31,黑龙江,齐齐哈尔市,食品饮料,2022,12,4,4161


<h1>Category filtering</h1>

In [7]:
l1=[]
l2=[]
for _idx, _group in df1.groupby('item_first_cate_name'):
    l1.append(_idx)
    l2.append(sum(_group['nums']))
temp_re=pd.DataFrame({'cate':l1, 'num':l2})
temp_re.sort_values('num', ascending=False, inplace=True)

In [8]:
temp_re1=temp_re[temp_re['num']>1000000]  #60 categories
features=list(temp_re1['cate'])
df1=df1[df1['item_first_cate_name'].isin(features)]

In [14]:
zhixia=['北京', '重庆', '上海', '天津']

In [15]:
df1_temp1=df1[~df1['rev_addr_province_id'].isin(zhixia)].copy()
df1_temp2=df1[df1['rev_addr_province_id'].isin(zhixia)].copy()
df1_temp2.rev_addr_city_id=df1_temp2.rev_addr_province_id
df1_temp2 = df1_temp2.groupby(['sale_ord_dt','rev_addr_province_id','rev_addr_city_id','item_first_cate_name','year','month','season'],as_index=False).sum()
df1_temp1 = df1_temp1.groupby(['sale_ord_dt','rev_addr_province_id','rev_addr_city_id','item_first_cate_name','year','month','season'],as_index=False).sum()

In [16]:
df1=pd.concat([df1_temp1,df1_temp2])
df1=df1.reset_index(drop=True)

In [18]:
df1['rev_addr_city_id'] = df1['rev_addr_city_id'].str.replace('北京', '北京市')
df1['rev_addr_city_id'] = df1['rev_addr_city_id'].str.replace('重庆', '重庆市')
df1['rev_addr_city_id'] = df1['rev_addr_city_id'].str.replace('上海', '上海市')
df1['rev_addr_city_id'] = df1['rev_addr_city_id'].str.replace('天津', '天津市')

In [20]:
df2=df1.groupby(['rev_addr_city_id','item_first_cate_name','year'],as_index=False)['nums'].sum()

In [23]:
l_filter=['东丽区','东城区','丰台区','北碚区','和平区','嘉定区','大兴区','宁河区','宝山区', '崇明区','延庆区','徐汇区',
         '普陀区', '朝阳区', '杨浦区', '松江区','梁平区', '汉沽区', '津南区','浦东新区','渝北区','滨海新区','石景山区',
         '红桥区','虹口区','西青区','金山区', '长宁区', '闵行区','青浦区','静安区','黄浦区', '丰都县', '城口县','巫山县','石柱县']

In [24]:
df2=df2[~df2['rev_addr_city_id'].isin(l_filter)]

In [25]:
l_city=df2.rev_addr_city_id.unique()  #359个城市

In [28]:
df2.to_csv('filter_data.csv', index=False)

In [29]:
data_all = pd.read_csv('filter_data.csv')

data_all = data_all[data_all['year'] != 2023]
data_all = data_all[data_all['year'] != 2020]

data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['纸品家清'], '家庭清洁/纸品')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['鞋靴','鞋包服饰/百货'], '服饰内衣')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['医药保健','营养保健','传统滋补'], '医疗保健')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['摩托车/电动车'], '汽车用品')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['宠物健康'], '宠物生活')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['鲜花/奢侈品'], '珠宝首饰')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['钟表'], '钟表眼镜')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['手机数码'], '数码')
data_all['item_first_cate_name'] = data_all['item_first_cate_name'].replace(['个护美妆','美妆护肤'],'个人护理')

data_all = data_all.reset_index(drop=True)
data_all.to_csv('filter_data_2.csv', index=False)